In [ ]:
import os
import arcpy
arcpy.env.overwriteOutput = True

hex_level = 'hexbin_h3_r0'
directory = r"E:\gis_lib\CMIP6\Ocean CMIP6\Ocean CRFs"
rasters = [f for f in os.listdir(directory) if f.endswith('.crf') and os.path.isdir(os.path.join(directory, f))]
SDE = r"E:\gis_projects\Ocean and Coastal Information System\sde_on_OCIS.sde"
stats = 'SUM'
stats = 'MIN, MAX, MEAN'
print(rasters)

In [14]:
## This function summarizes a non-multidimensional raster using SUM statistics
def hexbin_summary(hex_level, rasters, stats):
    for raster in rasters:
            print(raster)
            raster_name = raster.split('.')[0]
            print(raster_name)
            SDE_Path = SDE
            table_name = os.path.join(SDE_Path, raster_name + hex_level)
            with arcpy.EnvManager(cellSize="MINOF"):
                arcpy.ia.ZonalStatisticsAsTable(
                    in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
                    zone_field="grid_id",
                    in_value_raster=os.path.join(directory, raster),
                    out_table=os.path.join(SDE_Path, raster_name + hex_level),
                    ignore_nodata="DATA",
                    statistics_type=stats,
                    process_as_multidimensional="CURRENT_SLICE",
                    percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",
                    circular_calculation="ARITHMETIC",
                    circular_wrap_value=360,
                    out_join_layer=None
                )
            arcpy.management.AlterField(
            in_table=table_name,
            field=stats,
            new_field_name=raster_name[:20].lower()+"_"+stats.lower(),
            new_field_alias=raster_name.lower()+"_"+stats.lower(),
            field_type="",
            field_length=8,
            field_is_nullable="NULLABLE",
            clear_field_alias="DO_NOT_CLEAR"
            )

In [15]:
## This function summarizes a multidimensional raster using MIN_MAX_MEAN statistics
def hexbin_summary_md(hex_level, rasters, stats):
    for raster in rasters:
        print(raster)
        raster_name = raster.split('.')[0]
        print(raster_name)
        SDE_Path = SDE
        table_name = os.path.join(SDE_Path, raster_name + hex_level)
        with arcpy.EnvManager(cellSize="MINOF"):
            arcpy.ia.ZonalStatisticsAsTable(
                in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
                zone_field="grid_id",
                in_value_raster=os.path.join(directory, raster),
                out_table=os.path.join(SDE_Path, raster_name + hex_level),
                ignore_nodata="DATA",
                statistics_type="MIN_MAX_MEAN",
                process_as_multidimensional="CURRENT_SLICE",
                percentile_values=[90],
                percentile_interpolation_type="AUTO_DETECT",
                circular_calculation="ARITHMETIC",
                circular_wrap_value=360,
                out_join_layer=None
            )
        arcpy.management.AlterField(
        in_table=table_name,
        field="min",
        new_field_name=raster_name[:20].lower()+"_min",
        new_field_alias=raster_name.lower()+"_min",
        field_type="",
        field_length=8,
        field_is_nullable="NULLABLE",
        clear_field_alias="DO_NOT_CLEAR"
        )
        arcpy.management.AlterField(
        in_table=table_name,
        field="max",
        new_field_name=raster_name[:20].lower()+"_max",
        new_field_alias=raster_name.lower()+"_max",
        field_type="",
        field_length=8,
        field_is_nullable="NULLABLE",
        clear_field_alias="DO_NOT_CLEAR"
        )
        arcpy.management.AlterField(
        in_table=table_name,
        field="mean",
        new_field_name=raster_name[:20].lower()+"_mean",
        new_field_alias=raster_name.lower()+"_mean",
        field_type="",
        field_length=8,
        field_is_nullable="NULLABLE",
        clear_field_alias="DO_NOT_CLEAR"
        )


In [ ]:
hex_level = 'hexbin_h3_r0'
hexbin_summary_md(hex_level, rasters, stats)